### Get data from firebase.

In [48]:
# get firebase data
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import numpy as np
import pandas as pd
import math

In [4]:
cred = credentials.Certificate('../../credentials.json')
default_app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [5]:
gt_ref = db.collection(u'gt')

In [22]:
bike_thefts_firebase = gt_ref.where('OffenseCode', "==", '2317')

### Load into pandas dataframe

The keys from firebase: dict_keys(['LocationLandmark', 'CreatedSource', 'IncidentToDate', 'OCANumber', 'Offense Description', 'CaseStatus', 'OffenseCode', 'CaseDisposition', 'LocationCoordinates', 'LocationCode', 'IncidentFromTime', 'LocationDirectional', 'LocationStreet', 'LocationStreetNumber', 'PatrolZone', 'IncidentFromDate', 'IncidentToTime'])

TODO: convert to getting data into firebase rather than from files manually

In [79]:
d = bike_thefts_firebase.get()

a1 = []
good = 0
total = 0
for res in d:
    
    item = res.to_dict()
    try:
        print('{}, {}'.format(item['LocationCoordinates'].latitude, item['LocationCoordinates'].longitude))    
        good += 1
    except AttributeError:
        # print('null')
        pass
    
    total += 1

print('Total')

33.7855073, -84.3926512
33.7794998804019, -84.40233694088559
33.7778625857, -84.40391085679998
33.7789564125, -84.4013146596
33.77279729, -84.39162219
33.7791488586, -84.40394356849998
33.7794998804019, -84.40233694088559
33.77817418, -84.40548218
33.7755668, -84.40420929999999
33.7758956, -84.3916764
33.7820202, -84.39637350000001
33.7789337787049, -84.4061169086705
33.7800512398, -84.4019878567
33.7819602, -84.3954325
33.77608578, -84.39608548
33.77551971, -84.39447575
33.77895641, -84.40131466
33.7791442172, -84.4035824555
33.782380022800005, -84.3959555597
33.77628053, -84.38801865
33.7721292, -84.3909436
33.7762805252, -84.3880186544
33.77597531, -84.39538136
33.7749203, -84.3964148
33.7796545110447, -84.40422607259809
33.7794998804019, -84.40233694088559
33.7749203, -84.3964148
33.7741835, -84.3987867
33.77608578, -84.39608548
33.77450626, -84.39105819
33.7867931923, -84.4062145431
33.7700594, -84.39162449999999
33.7760857762, -84.39608548399998
33.776904, -84.3931271
33.77191256

In [80]:
print('Number of good records: ', good)

Number of good records:  208


### Get only bike thefts from csv files

In [63]:
# Load & conjoin
GTPD_FILES = ['crimelog2013.csv', 'crimelog2014.csv', 'crimelog2015.csv', 'crimelog2016.csv', 'crimelog2017.csv', 'crimelog2018.csv']
DATA_PREFIX = '../../data/'
dataGT = pd.DataFrame() 
concatter2 = [] 
for file in GTPD_FILES: 
    tmp_data = pd.read_csv(DATA_PREFIX + file)
    concatter2.append(tmp_data)   

dataGT = pd.concat(concatter2, axis=0)

# theres about 43 out of 11876 rows with a NaN offense code so we delete those
dataGT = dataGT[pd.notnull(dataGT['OffenseCode'])]

In [66]:
bike_data = dataGT.loc[dataGT['OffenseCode'] == '2317']

In [74]:
bike_data['LocationLatitude'].isna().sum()

99

In [85]:
cleaned_bike = bike_data[['IncidentFromDate', 'IncidentFromTime', 'LocationLatitude', 'LocationLongitude']]

cleaned_bike.dropna()

,IncidentFromDate,IncidentFromTime,LocationLatitude,LocationLongitude
278,02/06/2013,23:00:00,33.7769,-84.392712
442,02/26/2013,21:00:00,33.7797,-84.404226
692,04/02/2013,21:00:00,33.7713,-84.391332
1044,05/21/2013,22:00:00,33.7795,-84.402337
1060,05/05/2013,17:00:00,33.7773,-84.403867
1120,05/31/2013,19:00:00,33.7713,-84.391332
1264,06/27/2013,12:30:00,33.7815,-84.400140
1363,07/15/2013,16:30:00,33.7713,-84.393719
1480,08/09/2013,02:00:00,33.7826,-84.391641
1483,08/09/2013,17:45:00,33.7862,-84.404965
